Selleks, et käivitada järgnevad märkmikud, peate esmalt juurutama mudeli, mis kasutab baasmudelina `text-embedding-ada-002`, ja määrama juurutamise nime `.env` failis kui `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`, kui te pole seda veel teinud.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Järgmisena laadime Embedding Indexi Pandase andmeraami. Embedding Index on salvestatud JSON-faili nimega `embedding_index_3m.json`. Embedding Index sisaldab YouTube'i transkriptsioonide sisseehitusi kuni 2023. aasta oktoobri lõpuni.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Järgmisena loome funktsiooni nimega `get_videos`, mis otsib päringu jaoks Embedding Indexist. Funktsioon tagastab 5 videot, mis on päringule kõige sarnasemad. Funktsioon töötab järgmiselt:

1. Kõigepealt luuakse Embedding Indexi koopia.
2. Seejärel arvutatakse päringu Embedding, kasutades OpenAI Embedding API-d.
3. Seejärel lisatakse Embedding Indexisse uus veerg nimega `similarity`. Veerg `similarity` sisaldab kosinussarnasust päringu Embeddingi ja iga videosegmendi Embeddingi vahel.
4. Järgmisena filtreeritakse Embedding Index veeru `similarity` alusel. Embedding Index filtreeritakse nii, et see sisaldaks ainult videoid, mille kosinussarnasus on vähemalt 0.75.
5. Lõpuks sorteeritakse Embedding Index veeru `similarity` järgi ja tagastatakse 5 parimat videot.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

See funktsioon on väga lihtne, see lihtsalt kuvab otsingupäringu tulemused.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Kõigepealt laaditakse Embedding Index Pandase DataFrame'i.
2. Seejärel palutakse kasutajal sisestada päring.
3. Seejärel kutsutakse `get_videos` funktsioon, et otsida päringule vastavaid tulemusi Embedding Indexist.
4. Lõpuks kutsutakse `display_results` funktsioon, et kuvada tulemused kasutajale.
5. Seejärel palutakse kasutajal sisestada uus päring. See protsess kordub, kuni kasutaja sisestab `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.et.png)

Teilt küsitakse päringut. Sisestage päring ja vajutage Enter. Rakendus tagastab nimekirja videotest, mis on päringuga seotud. Rakendus tagastab ka lingi video sellele kohale, kus küsimusele vastus asub.

Siin on mõned päringud, mida proovida:

- Mis on Azure Machine Learning?
- Kuidas töötavad konvolutsioonilised närvivõrgud?
- Mis on närvivõrk?
- Kas ma saan kasutada Jupyter Notebooks'i koos Azure Machine Learninguga?
- Mis on ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
